# Static Website on S3 with boto3

In [1]:
import boto3
from botocore.exceptions import ClientError

import logging
import os

## Why this project

As a Data Scientist, I do care that my projects are running in production, therefore my interest in Data Engineering and DevOps part. I also care, that business-users have access to insights and advanced visualizations that I produce. 

Therefore I make those visualizations, including interactive visualizations, with Jupyter notebooks. They are easily converted to HTML files, which can constitute static website. And I host those websites. 

AWS S3 is a perfect opportunity to host everything simple and quickly. I am just starting with boto3 and I love it. No mouse-clicking, just python script. 

## Purpose

Here I will create a static Website using AWS S3 bucket with 'boto3' - python SDK that AWS offers. The content includes several/many files, all of which are locally saved in one _nested_ directory.

## Get access to AWS

First of all I need to get access to my AWS account.

A lot of tutorials explain at this place, how to use your 'Access key ID' and your 'Secret access key' without exposing them in the code. 

But we  are on 'Cloud Track' of Bertelsmann ..., aren't we ? We have learned the better way - creating Programmatic User at AWS. Actually we already created such a user during the course. Let us use it!

With Programmatic User, the credentials are saved on your computer in ~/.aws/credentials file.
Just choose the one of the profiles in this file. Then use the name of this profile (here 'default') to creating a boto3 session. 

In [2]:
session = boto3.Session(profile_name='default')

You also do not need to manually enter your region, it is available from session. 

In [3]:
current_region = session.region_name
print(current_region)

us-west-2


## Client and Resource

boto3 has _client_ - and _resource_ -level access to AWS. See [a short explanation here](https://python-decompiler.com/article/2017-03/difference-in-boto3-between-resource-client-and-session) 

My purpose is to use _resource_ level only. I use the [documentation here](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3.html#service-resource)

For people interested in _client_ level, I recommend the [GitHub of Balazs Kocsis](https://github.com/bkocis/bertelsmann-dsml-group-projects/blob/master/Project-boto/AWS-SDK-for-python-boto3.ipynb)

First I create _resource_.


In [4]:
s3_resource = boto3.resource('s3')

## Create bucket and upload files

Before actually starting with bucket and file uploading, let me introduce the `mimetypes` python module. 
Just have a look at the dictionary, mapping filename extensions to MIME types. 

In [7]:
import mimetypes
for fileext in ['.html', '.png']:
    print('For file extension {:s} --> the MIME type is {:s}'.format(fileext, mimetypes.types_map[fileext]))

For file extension .html --> the MIME type is text/html
For file extension .png --> the MIME type is image/png


In order to be able to deal with unknown types, I convert it to _defaultdict_ with the default MIME type being 'application/octet-stream' 

In [8]:
from collections import defaultdict
content_type = defaultdict(lambda: 'application/octet-stream', mimetypes.types_map)
for fileext in ['.html', '.png', '.coffee']:
    print('For file extension {:s} --> the MIME type is {:s}'.format(fileext, content_type[fileext]))

For file extension .html --> the MIME type is text/html
For file extension .png --> the MIME type is image/png
For file extension .coffee --> the MIME type is application/octet-stream


Why care about MIME type? Well, you need this to display your HTML file (in your website) in browser.  When I uploaded files without specifying the MIME type explicitly, my files were not displayed. Instead I got an option to download the file.

The content of my website is stored in on nested directory. So, I want to upload all the files from this directory. In AWS CLI I found the `--recursive` flag for this purpose. But how to do this with boto3?
I only found the option to iterate over all the files. 

Now we are ready to create the bucket and upload the content of `target_dir`.

In [7]:
target_dir = 'wholesale-more-frozen-products'
bucket_name = target_dir

In [46]:
# ----- test structure of nested directory
# target_dir = bucket_name
# for subdir, dirs, files in os.walk(target_dir):
#     print(subdir)
#     #print(dirs)
#     for file in files:
#         print("\t", os.path.join(subdir, file)) #.replace(target_dir+'/', '')

In [ ]:
def create_bucket(s3_resource, aws_region, bucket_name):
    try:
        s3_resource.create_bucket(Bucket=bucket_name,
                         CreateBucketConfiguration={'LocationConstraint': aws_region})
    except ClientError as err:
        print(err)
    
    return None

In [38]:
def upload_nested_directory(s3_resource, aws_region, bucket_name, target_dir):
    if not os.path.isdir(target_dir):
        raise ValueError('target_dir %r not found.' % target_dir)
        
    for subdir, dirs, files in os.walk(target_dir):
        for file in files:
            filename = os.path.join(subdir, file)
            s3_path = filename.replace(target_dir+'/', '')
            fileext = '.'+file.split('.')[-1]  # file extension
            s3_resource.Object(bucket_name, s3_path).put(Body=open(filename, 'rb'),
                                                          ACL = 'public-read',
                                                          ContentType = content_type[fileext])
    
            print('File uploaded to https://s3.%s.amazonaws.com/%s/%s' % (
                                                    aws_region, bucket_name, s3_path))
    return None

In [14]:
create_bucket(s3_resource=s3_resource, aws_region=current_region, bucket_name=bucket_name)

In [39]:
upload_nested_directory(s3_resource=s3_resource, aws_region=current_region, 
                        bucket_name=bucket_name, target_dir=target_dir)

File uploaded to https://s3.us-west-2.amazonaws.com/wholesale-more-frozen-products/frozen3.html
File uploaded to https://s3.us-west-2.amazonaws.com/wholesale-more-frozen-products/frozen.html
File uploaded to https://s3.us-west-2.amazonaws.com/wholesale-more-frozen-products/wholesale-moreFrozenProducts.html
File uploaded to https://s3.us-west-2.amazonaws.com/wholesale-more-frozen-products/wholesale-moreFrozen_files/figure-html/network_continuous-1.png
File uploaded to https://s3.us-west-2.amazonaws.com/wholesale-more-frozen-products/wholesale-moreFrozen_files/figure-html/network_all-1.png
File uploaded to https://s3.us-west-2.amazonaws.com/wholesale-more-frozen-products/me_assets/d3.v3.4.3.js
File uploaded to https://s3.us-west-2.amazonaws.com/wholesale-more-frozen-products/me_assets/d3_tip.js
File uploaded to https://s3.us-west-2.amazonaws.com/wholesale-more-frozen-products/me_assets/chart.css
File uploaded to https://s3.us-west-2.amazonaws.com/wholesale-more-frozen-products/me_assets/

Let us list all files in the bucket to ensure that everyting is uploaded properly

## Get bucket subresource

In [8]:
bucket = s3_resource.Bucket(bucket_name)
for obj in bucket.objects.all():
    print(obj.key, obj.last_modified)

frozen.html 2020-02-06 14:49:07+00:00
frozen3.html 2020-02-06 14:49:06+00:00
me_assets/Points/plot.coffee 2020-02-06 14:49:14+00:00
me_assets/Points/plot.js 2020-02-06 14:49:14+00:00
me_assets/chart.css 2020-02-06 14:49:11+00:00
me_assets/d3.v3.4.3.js 2020-02-06 14:49:11+00:00
me_assets/d3_tip.css 2020-02-06 14:49:14+00:00
me_assets/d3_tip.js 2020-02-06 14:49:11+00:00
me_assets/me.coffee 2020-02-06 14:49:12+00:00
me_assets/me.css 2020-02-06 14:49:12+00:00
me_assets/me.js 2020-02-06 14:49:13+00:00
wholesale-moreFrozenProducts.html 2020-02-06 14:49:08+00:00
wholesale-moreFrozen_cache/html/__packages 2020-02-06 14:49:15+00:00
wholesale-moreFrozen_cache/html/set-options_18066191f72c912163595330ff610c0c.RData 2020-02-06 14:49:16+00:00
wholesale-moreFrozen_cache/html/set-options_18066191f72c912163595330ff610c0c.rdb 2020-02-06 14:49:17+00:00
wholesale-moreFrozen_cache/html/set-options_18066191f72c912163595330ff610c0c.rdx 2020-02-06 14:49:17+00:00
wholesale-moreFrozen_cache/html/unnamed-chunk-

In [9]:
index_document = 'wholesale-moreFrozenProducts.html'

Now I can access my files and see the content by using the http path:

In [10]:
print("http://{}.s3.amazonaws.com/{}".format(bucket_name, index_document))

http://wholesale-more-frozen-products.s3.amazonaws.com/wholesale-moreFrozenProducts.html


Try to use the link above. It works for me. Does it for you?

Are we done? Is this my website? The point of calling it 'website' is that you do not need to use the filename directly. The website should know its index page; in other words, it should have the endpoint. So I need to set the website configuration first.

## Error and Index for Website

Here I am specifying the _index_ and _error_ documents of my website. 

In [35]:
bucket_website = s3_resource.BucketWebsite(bucket_name)

In [36]:
website_configuration = {
    'ErrorDocument': {'Key': 'error.html'},
    'IndexDocument': {'Suffix': index_document},
}

In [37]:
bucket_website.put(WebsiteConfiguration=website_configuration)

{'ResponseMetadata': {'RequestId': '2017BCCA2F64585F',
  'HostId': '4SpZYbrX7mngItpxtg3gJMAaIs7wcfrX1kCOdcyhiit92MvZcExCte7t4zOXHt81c1TU8EA/S9A=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '4SpZYbrX7mngItpxtg3gJMAaIs7wcfrX1kCOdcyhiit92MvZcExCte7t4zOXHt81c1TU8EA/S9A=',
   'x-amz-request-id': '2017BCCA2F64585F',
   'date': 'Fri, 07 Feb 2020 08:43:20 GMT',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

To retrieve configuration use

In [38]:
print(bucket_website.error_document)
print(bucket_website.index_document)

{'Key': 'error.html'}
{'Suffix': 'wholesale-moreFrozenProducts.html'}


I did not found a way to get the endpoint of the website programatically. Let us construct it manually

In [14]:
print("http://{}.s3-website-{}.amazonaws.com/".format(bucket_name, current_region))

http://wholesale-more-frozen-products.s3-website-us-west-2.amazonaws.com/


Now I get access to the website without specifying the index document.

## Delete Website configuration

To save AWS resources, delete the website.

In [34]:
bucket_website.delete()

{'ResponseMetadata': {'RequestId': 'F52F13E197B21E6D',
  'HostId': 'DUKw4kLzvnOEtwC4C2ZE+E0LjbrAo0KQizlmhgIxtV56Z5YYyj+pFX+fT0PR8MJ+r0AUZt+2qS4=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': 'DUKw4kLzvnOEtwC4C2ZE+E0LjbrAo0KQizlmhgIxtV56Z5YYyj+pFX+fT0PR8MJ+r0AUZt+2qS4=',
   'x-amz-request-id': 'F52F13E197B21E6D',
   'date': 'Fri, 07 Feb 2020 08:42:42 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

Since my files are still in the bucket, I can access them directly by name

In [ ]:
print("http://{}.s3.amazonaws.com/{}".format(bucket_name, index_document))

## Delete files in the bucket

Now I can delete all files in the bucket

In [ ]:
for obj in bucket.objects.all():
    obj.delete()

## Delete Bucket

As soon as the bucket is empty, I can delete it

In [ ]:
response = bucket.delete()